<a href="https://colab.research.google.com/github/charlieojackson/weighted_average_rank/blob/main/Weighted_Average_Rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Weighted average rank

## Weighted average in three steps
1. Determine the weight percetange
2. Multiply the value by the weight percentage
3. Sum the weighted values

In [ ]:
df = pd.DataFrame({'keyword':['sofa', 'sofa grey', 'sofa red'], 'avg_sv':[1000,100,10], 'pos':[1, 10, 20]})

In [ ]:
df

,keyword,avg_sv,pos
0,sofa,1000,1
1,sofa grey,100,10
2,sofa red,10,20


In [ ]:
# 1. Determine the weight 
df['weight_pct'] = df['avg_sv'] / df['avg_sv'].sum()

In [ ]:
# 2. Multiply the weight by each value
df['weight_actual'] = df['pos'] * df['weight_pct']

In [ ]:
# 3. Sum the weights
df['weight_actual'].sum()

1.9819819819819817

In [ ]:
df

,test,avg_sv,pos,weight_pct,weight_actual
0,sofa,1000,1,0.900901,0.900901
1,sofa grey,100,10,0.090090,0.900901
2,sofa red,10,20,0.009009,0.180180


## Putting it together into a function

In [ ]:
# put into a function
def add_weight_column(pos, weight):
  weight_pct    = weight / df['avg_sv'].sum()
  weight_actual = weight_pct * pos
  return [weight_pct, weight_actual]

In [ ]:
for index, row in df.iterrows():
  pos    = row['pos']
  weight = row['avg_sv']

  weight_pct, weight_actual = add_weight_column(pos, weight)
  df.loc[index,'weight_pct']    = weight_pct
  df.loc[index,'weight_actual'] = weight_actual

In [ ]:
df

,test,avg_sv,pos,weight_pct,weight_actual
0,sofa,1000,1,0.900901,0.900901
1,sofa grey,100,10,0.090090,0.900901
2,sofa red,10,20,0.009009,0.180180


In [ ]:
df.weight_actual.sum()

1.9819819819819817

# Weighted average with groupby

In [ ]:
df = pd.DataFrame({'keyword':['table', 'wood table', 'round table', 'sofa grey', '2 seater sofa', 'wooden chair', 'rocking chair'], 'group':['table', 'table','table', 'sofa', 'sofa', 'chair', 'chair'], 'avg_sv':[1000,100,10, 200, 100, 500, 1000], 'pos':[1, 10, 20, 15, 11, 16, 9]})

In [ ]:
df

,keyword,group,avg_sv,pos
0,table,table,1000,1
1,wood table,table,100,10
2,round table,table,10,20
3,sofa grey,sofa,200,15
4,2 seater sofa,sofa,100,11
5,wooden chair,chair,500,16
6,rocking chair,chair,1000,9


In [ ]:
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
df.groupby('group').apply(wavg, 'pos', 'avg_sv')

group
chair    11.333333
sofa     13.666667
table     1.981982
dtype: float64